In [ ]:
import requests
import pandas as pd

API_KEY = "your_key_here"
specs_url = f"https://api.edmunds.com/api/vehicle/v2/makes?state=used&view=full&fmt=json&api_key={API_KEY}"
specs_data = requests.get(specs_url).json()['makes']
specs_df = pd.json_normalize(specs_data)

In [ ]:
historical_url = f"https://api.edmunds.com/api/v2/historicalcost?make={make}&model={model}&year={year}&api_key={API_KEY}"
historical_data = requests.get(historical_url).json()
historical_df = pd.json_normalize(historical_data)

In [ ]:
combined_df = pd.merge(
    specs_df[['make.name', 'model.name', 'year.year', 'engine.horsepower', 'price.baseMSRP']],
    historical_df[['averagePrice', 'percentDepreciation', 'mileageBand']],
    left_on=['make.name', 'model.name', 'year.year'],
    right_on=['make', 'model', 'year'],
    how='left'
)

In [6]:
import requests

url = "https://cars-by-api-ninjas.p.rapidapi.com/v1/cars"

querystring = {"model":"corolla"}

headers = {
	"x-rapidapi-key": "003deefbf4mshf3349f539514267p174164jsn601b560afa7d",
	"x-rapidapi-host": "cars-by-api-ninjas.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

print(response.json())

[{'city_mpg': 'this field is for premium subscribers only', 'class': 'compact car', 'combination_mpg': 'this field is for premium subscribers only', 'cylinders': 4, 'displacement': 1.6, 'drive': 'fwd', 'fuel_type': 'gas', 'highway_mpg': 'this field is for premium subscribers only', 'make': 'toyota', 'model': 'corolla', 'transmission': 'a', 'year': 1993}]


In [34]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
import re

# Configuration
CHROME_DRIVER_PATH = "/path/to/chromedriver"  # Download from https://chromedriver.chromium.org/
BASE_URL = "https://www.cardekho.com/volkswagen"

def setup_driver():
    """Configure headless Chrome browser"""
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")
    service = Service(CHROME_DRIVER_PATH)
    driver = webdriver.Chrome(service=service, options=chrome_options)
    return driver

def scrape_cardekho_vw(driver, model="tiguan"):
    """Scrape specific Volkswagen model data from CarDekho"""
    print(f"Scraping Volkswagen {model}...")
    driver.get(f"{BASE_URL}/{model}")
    time.sleep(5)  # Wait for page load
    
    # Scroll to load all listings
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    cars = []
    
    for listing in soup.select('div.gsc_col-xs-12.gsc_col-sm-12.gsc_col-md-8.gsc_col-lg-8'):
        try:
            car_data = {
                'model': model.upper(),
                'variant': listing.find('h3', class_='variantName').text.strip(),
                'price': listing.find('div', class_='price').text.strip().replace('₹', '').replace(',', ''),
                'year': re.search(r'\d{4}', listing.find('div', class_='holder').text).group(0),
                'mileage': listing.find('li', title='Mileage').text.replace(' kmpl', '') if listing.find('li', title='Mileage') else None,
                'engine': listing.find('li', title='Engine').text if listing.find('li', title='Engine') else None,
                'transmission': listing.find('li', title='Transmission').text if listing.find('li', title='Transmission') else None,
                'link': "https://www.cardekho.com" + listing.find('a')['href']
            }
            cars.append(car_data)
        except Exception as e:
            print(f"Skipped listing: {str(e)}")
            continue
    
    return pd.DataFrame(cars)

# Main execution
driver = setup_driver()
try:
    # Scrape multiple VW models
    models = ["tiguan", "vento", "taigun", "virtus", "polo"]
    all_data = pd.DataFrame()
    
    for model in models:
        model_df = scrape_cardekho_vw(driver, model)
        all_data = pd.concat([all_data, model_df], ignore_index=True)
        time.sleep(5)  # Be polite between requests
    
    # Clean data
    all_data['price'] = pd.to_numeric(all_data['price'], errors='coerce')
    all_data['mileage'] = pd.to_numeric(all_data['mileage'], errors='coerce')
    
    # Save to CSV
    filename = f"volkswagen_cardekho_{pd.Timestamp.now().strftime('%Y%m%d')}.csv"
    all_data.to_csv(filename, index=False)
    print(f"\n✅ Collected {len(all_data)} Volkswagen listings from CarDekho")
    print(all_data.head().to_markdown())
    
finally:
    driver.quit()

NoSuchDriverException: Message: Unable to obtain driver for chrome; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors/driver_location
